# 1. Đăng nhập 

In [ ]:
import pandas as pd
from FiinQuantX import FiinSession
import json
from datetime import datetime
# --- 1. Đăng nhập ---
username = 'DSTC_38@fiinquant.vn'
password = 'Fiinquant0606'
client = FiinSession(username=username, password=password).login()

# 2. Crawl Data from 2022 - 2025 

In [ ]:
# --- Lấy danh sách mã VN30 và VNMid---
tickers_vn30 = list(client.TickerList(ticker="VN30"))
tickers_vnmid = list(client.TickerList(ticker="VNMid"))
tickers = tickers_vn30 + tickers_vnmid

data = client.Fetch_Trading_Data(
    realtime=False,
    tickers=tickers,
    fields = ['open', 'high', 'low', 'close', 'volume', 'bu','sd', 'fb', 'fs', 'fn'], 
    adjusted=True,
    by="1d",
    from_date="2019-01-01",
    to_date=datetime.today().strftime("%Y-%m-%d") 
).get_data()

# 3. Tính các chỉ báo: 

In [ ]:
fi = client.FiinIndicator()

# RSI (14 ngày)
data['RSI_14'] = fi.rsi(data['close'], window=14)

# MA20, MA50, MA200
data['MA20'] = fi.sma(data['close'], window=20)
data['MA50'] = fi.sma(data['close'], window=50)
data['MA200'] = fi.sma(data['close'], window=200)

# ATR (20 ngày)
data['ATR20'] = fi.atr(data['high'], data['low'], data['close'], window=20)

# Volume trung bình 20 ngày
data['Vol_Avg20'] = fi.sma(data['volume'], window=20)

print(data.tail(30))  
data.to_csv("data.csv")

# 4. Crawl Financial Report from 2022 to 2025

In [ ]:
fa = client.FundamentalAnalysis()
all_ratios = []
for year in range(2019, datetime.now().year + 1):
    ratios = fa.get_ratios(
        tickers=tickers,
        TimeFilter="Quarterly",   
        NumberOfPeriod=4,         
        LatestYear=year,
        Consolidated=True
    )
    all_ratios.extend(ratios)
df = pd.DataFrame(all_ratios)
print(df.head(20))
print("Số dòng:", len(df))
df.to_csv("fundamental_ratios.csv", index=False, encoding="utf-8-sig")
print("✅ Đã lưu fundamental_ratios.csv từ 2019 đến nay")

# 5. Parsing to take out indicators: PE, PB, ROE, EPS,....

In [ ]:
import pandas as pd
import ast

def extract_ratios(df: pd.DataFrame, drop_empty: bool = True) -> pd.DataFrame:
    # Chuyển cột ratios từ string sang dict, bỏ qua giá trị trống
    def safe_eval(x):
        if pd.isna(x) or str(x).strip() == "":
            return {}
        return ast.literal_eval(x)

    df['ratios'] = df['ratios'].apply(safe_eval)
    if drop_empty:
        df = df[df['ratios'].apply(lambda r: len(r) > 0)].reset_index(drop=True)
    def flatten_ratios(ratios_dict):
        flat = {}
        for category, metrics in ratios_dict.items():
            for key, value in metrics.items():
                if key == "BasicEPS":
                    flat["EPS"] = value
                elif key == "PriceToBook":
                    flat["PB"] = value
                elif key == "PriceToEarning":
                    flat["PE"] = value
                else:
                    flat[key] = value
        return flat

    ratios_expanded = df['ratios'].apply(flatten_ratios).apply(pd.Series)
    df_final = pd.concat([df.drop(columns=['ratios']), ratios_expanded], axis=1)

    return df_final

df = pd.read_csv("fundamental_ratios.csv")
df_extracted = extract_ratios(df)

output_file = "fundamental_ratios_extracted.csv"
df_extracted.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ Đã lưu file kết quả tại: {output_file}")


# 6. Crawl VNINDEX

In [ ]:

vnindex = client.Fetch_Trading_Data(
    realtime=False,
    tickers=["VNINDEX"],  
    fields=["close"],
    adjusted=True,
    by="1d",
    from_date="2022-01-01",
    to_date=datetime.today().strftime("%Y-%m-%d")
).get_data()


vnindex["daily_return"] = vnindex["close"].pct_change()
vnindex["equity_curve"] = (1 + vnindex["daily_return"].fillna(0)).cumprod()


vnindex.to_csv("vnindex_equity.csv", index=False, encoding="utf-8-sig")

print("✅ Đã lưu file vnindex_equity.csv")
print(vnindex.head())
print(vnindex.tail())